# 1.4 WebMaps erstellen

In diesem Notebook wird der Datenbezug und die Darstellung in einfachen Karten thematisiert.

Wir verwenden dazu das Modul **Folium**, welches die Leaflet JavaScript API nutzt. 

Es wäre auch möglich OpenLayers zu verwenden, aber momentan ist Folium unter Python eleganter, d.h. mit wenig Code hat man schon eine Karte.


Zunächst erstellen wir den Order "daten". Wir benötigen diesen um unsere Daten zu speichern:

In [1]:
import os

if not os.path.exists("daten"):
    os.mkdir("daten")

nun ist der Ordner erstellt. Wir importieren nun folium:

In [2]:
import folium

In [3]:
m = folium.Map(location=[47.37825,8.5367835], zoom_start=16) # Zürich, Hauptbahnhof
m

### Verwendung des swisstopo WMTS Layers

Es ist auch möglich WMS und WMTS Layer zu verwenden, das heisst wir können auch z.B. Daten der swisstopo, oder von kantonen verwenden. Mehr über WMS/WMTS dann morgen.

In [4]:
m = folium.Map(location=[47.55655834594247, 7.592509335704225], tiles='', zoom_start=15)
 
folium.raster_layers.TileLayer(
    # 3857 ist der eps code
    tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
    attr="© swisstopo", # dies müssen wir gemäss Nutzungsbedingunen angeben
    name="pixelkarte-farbe",
    min_zoom=8,
    max_zoom=18,
    # wo x und y achse ist (je nach wmts anders)
    tms=False,
    overlay=False,
    control=False,
).add_to(m)

folium.LayerControl().add_to(m)

m

Wir können Karten als HTML exportieren:

In [5]:
m.save("daten/karte.html")

Marker hinzufügen:

In [6]:
m = folium.Map(location=[47.37825,8.5367835], tiles='', zoom_start=17)
 
folium.raster_layers.TileLayer(
    tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
    attr="(c) swisstopo",
    name="pixelkarte-farbe",
    min_zoom=8,
    max_zoom=18,
    tms=False,
    overlay=False,
    control=False,
).add_to(m)

folium.LayerControl().add_to(m)

# Marker

folium.Marker([47.37695,8.5387885], popup="Hotel <b>Schweizerhof</b><br/><br/>This hotel is located in the center").add_to(m)
folium.Marker([47.376386,8.5386506], popup="Hotel St. Gotthard").add_to(m)
m

Marker mit Farben und Fontawesome-Symbolen:

https://fontawesome.com/v5.15/icons?d=gallery&p=2&s=regular,solid&m=free

In [7]:
m = folium.Map(location=[47.37825,8.5367835], tiles='', zoom_start=17)
 
folium.raster_layers.TileLayer(
    tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
    attr="(c) swisstopo",
    name="pixelkarte-farbe",
    min_zoom=8,
    max_zoom=18,
    tms=False,
    overlay=False,
    control=False,
).add_to(m)

folium.LayerControl().add_to(m)

folium.Marker([47.37695,8.5387885], 
              popup="Hotel Schweizerhof",
              icon=folium.Icon(color="red", prefix="fa", icon="hotel")).add_to(m)

folium.Marker([47.376386,8.5386506], 
              popup="Hotel St. Gotthard",
              icon=folium.Icon(color="green", prefix="fa", icon="hotel")).add_to(m)

folium.Marker([47.376192, 8.540005], 
              popup="Hotel Townhouse", 
              icon=folium.Icon(color="blue", prefix="fa", icon="beer")).add_to(m)

m

## 1.4.1 Datensatz Bahntunnels der SBB herunterladen

Open Data SBB an: https://data.sbb.ch/explore/?sort=modified

* https://opendata.swiss/de/dataset/tunnel1
* https://data.sbb.ch/explore/dataset/tunnel/information/

Auf der Homepage der opendata.swiss sehen wir die Daten in 4 Formaten:

* JSON:  https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/geojson
* JSON:  https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/json 
* ZIP: https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/shp 
* CSV: https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/csv

Wobei die Datenformate schon mal falsch deklariert sind. Das erste ist nämlich ein GeoJSON und das ZIP enthält effektiv ein Shapefile.
Wir könnten das Shapefile herunterladen und in einem GIS anzeigen, aber wir wollen Python dazu verwenden.

Schauen wir uns mal zunächst das JSON File an. In Python können wir ein File ganz einfach mit dem Modul urllib herunterladen:

In [8]:
import urllib
import shutil

In [9]:
# JSON File herunterladen:
filename, headers = urllib.request.urlretrieve("https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/json", "daten/tunnel.json")

In [10]:
# GeoJSON File herunterladen: ( https://geojson.org/ )
filename, headers = urllib.request.urlretrieve("https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/geojson", "daten/tunnel.geojson")

In [11]:
# Shapefile (als zip) herunterladen und entpacken:
filename, headers = urllib.request.urlretrieve("https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/shp", "daten/tunnel.zip")
shutil.unpack_archive("daten/tunnel.zip", "daten/tunnel")

In [12]:
# CSV herunterladen:
filename, headers = urllib.request.urlretrieve("https://data.sbb.ch/api/v2/catalog/datasets/tunnel/exports/csv", "daten/tunnel.csv")

In [14]:
file = open("daten/tunnel.csv", encoding="utf-8")

for line in file:
    #print(line.strip())
    pass
    
file.close()

In [15]:
file = open("daten/tunnel.csv", encoding="utf-8")

next(file)
for line in file:
    data = line.strip().split(";")
    tunnelname = data[3]
    jahr = int(data[5])
    e = float(data[6])
    n = float(data[7])
    if jahr < 1860:
        print(tunnelname, jahr, e, n)
    
file.close()

Tunnel du Mont Sagne(007) 1859 2556091.729 1215161.848
Mühlefluhtunnel BL 1857 2631223.838 1249974.953
Koblenztunnel 1854 2660021.674 1273071.48
Täusitunnel 1859 2707736.667 1234375.513
Tunnel de Vauseyon (005) 1859 2560239.455 1204392.539
Tunnel de la Luche (008) 1859 2553022.477 1202728.131
Aarburgertunnel 1856 2635256.833 1241535.864
Wipkingertunnel 1856 2682685.269 1250459.117
Schloss Laufentunnel 1857 2688341.346 1281304.66
Tunnel du Mormont sud 1 1856 2532040.602 1168018.754
Tunnel du Gibet(010) 1859 2559308.927 1204255.26
Bucktentunnel 1857 2631018.939 1250669.491
Tunnel de la Sauge (009) 1859 2552595.235 1202452.835
Tunnel de Combe (Convers, 006) 1858 2554110.904 1216610.007
Burgdorfertunnel 1857 2614931.649 1212875.019
Hauensteintunnel 1857 2632189.32 1247858.519
Bühltunnel SG 1859 2735182.914 1219513.948
Stutztunnel 1859 2732353.476 1219778.634
Tunnel de St-Maurice (003) 1859 2566344.617 1118978.934
Tunnel du Mormont nord 2 1856 2531968.869 1168380.954


In [16]:
m = folium.Map(location=[47.37825,8.5367835], tiles='', zoom_start=7)
 
folium.raster_layers.TileLayer(
    tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
    attr="(c) swisstopo",
    name="pixelkarte-farbe",
    min_zoom=8,
    max_zoom=18,
    tms=False,
    overlay=False,
    control=False,
).add_to(m)

folium.LayerControl().add_to(m)


file = open("daten/tunnel.csv", encoding="utf-8")

next(file)
for line in file:
    data = line.strip().split(";")
    tunnelname = data[3]
    jahr = int(data[5])
    lnglat = data[10].split(",")   # eigentlich [8]...
    lat = float(lnglat[0])
    lng = float(lnglat[1])
    if jahr < 1860:
        folium.Marker([lat,lng], 
              popup=f"{tunnelname}<br/>{jahr}",
              icon=folium.Icon(color="red", prefix="fa", icon="star")).add_to(m)
    
file.close()


m.save("daten/tunnel.html")
m

### Verwenden von Marker Clustern

Wenn sehr viele Marker auf der Karte sind, können diese mit Marker-Cluster entschärft werden. Dies ist ein plugin von Folium, welches noch separat importiert werden muss.

Unterschied:<br/>
Der Marker wird nicht zur Karte hinzugefügt, sondern zum zuvor erstellten Marker Cluster. Mit einem WMTS Layer ist das ganze ein wenig komplexer, deshalb wird es hier weggelassen und Default OpenStreetMap verwendet

In [17]:
from folium.plugins import MarkerCluster

m = folium.Map(location=[47.37825,8.5367835], zoom_start=7)

marker_cluster = MarkerCluster(control=True).add_to(m)
folium.LayerControl().add_to(m)

file = open("daten/tunnel.csv", encoding="utf-8")

next(file)
for line in file:
    data = line.strip().split(";")
    tunnelname = data[3]
    jahr = int(data[5])
    lnglat = data[10].split(",")   # oder [8] wenn bereinigt
    lat = float(lnglat[0])
    lng = float(lnglat[1])
    folium.Marker([lat,lng], popup=f"{tunnelname}<br/>{jahr}", icon=folium.Icon(color="orange", prefix="fa", icon="star")).add_to(marker_cluster)
    
file.close()

m.save("daten/tunnel_markercluster.html")
m



### Mehr Informationen zu Folium

Weitere Beispiele zu Folium können hier gefunden werden:
https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/

Folium selbst ist OpenSource und kann auf GitHub hier gefunden werden: https://github.com/python-visualization/folium